In [27]:
import os
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from skimage import io, transform


%matplotlib inline

from IPython.display import display, HTML
display(HTML("<style>.container { width: 90% !important}; </style>"))

In [2]:
## TODO: Better use tf, as allows easier training and testing (also develop implementation with pytorch and pytorch lightning)

In [45]:
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np


# Some parameters:
# Can check for the mean and deviation: https://github.com/pytorch/examples/blob/97304e232807082c2e7b54c597615dc0ad8f6173/imagenet/main.py#L197-L198
mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
valid_size = 0.2
batch_size = 32

# TODO: Have to define image loader
transform_training = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(15),   
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(saturation=0.2, contrast=0.2, hue=0.2, brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)]
)

transform_testing = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(), 
    transforms.Normalize(mean, std)]
)

train_df = pd.read_csv('./data/train_processed.csv')
test_df = pd.read_csv('./data/test_processed.csv')

In [46]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(
            self.df.iloc[idx, 0]
        )
        
        image = io.imread(img_name)
        
        label = self.df.iloc[idx, 1]
        label = np.array([label])
        label = label.astype('float')
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [30]:
train_data = CustomDataset('./data/train_processed.csv', transform_training)
test_data = CustomDataset('./data/test_processed.csv', transform_testing)

In [31]:
# Now we are going to create the validation data

## TODO: Train test split indices stratified
num_train = len(train_df)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train)) # We obtain a list of values that we will make as an split
train_idx = indices[split:]
valid_idx = indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# And now we create the loders
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

loaders_scratch = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

print(len(train_loader), len(valid_loader), len(test_loader))


280 70 150


In [33]:
for data in train_loader:
    print(data)

TypeError: Unexpected type <class 'numpy.ndarray'>

In [20]:
import torch.nn as nn
from torch import optim

use_cuda = torch.cuda.is_available()

criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    # The modoel has proved to learn better with Adam
    return optim.Adam(model.parameters(), lr=0.0003)

In [21]:
import torch.nn as nn
import torch.nn.functional as F

number_classes = len(train_df.label.unique())

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv_1 = nn.Conv2d(3, 16, 3, stride=1, padding=1) 
        self.conv_2 = nn.Conv2d(16, 32, 3, stride=1, padding=1) 
        self.conv_3 = nn.Conv2d(32, 64, 3, stride=1, padding=1) 
        self.conv_4 = nn.Conv2d(64, 128, 3, stride=1, padding=1) 
        self.conv_5 = nn.Conv2d(128, 256, 3, stride=1, padding=1) 

        
        # And also define a max pooling layer
        self.pool = nn.MaxPool2d(2,2)        
        
        # And linear models
        self.fc1 = nn.Linear(7*7*256, 1024) # 7 as (224/(2*2*2*2*2) and 128 as it is the depth
        # self.fc2 = nn.Linear(512, number_classes)

        # Other model that i tested
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, number_classes)

        
        # Will define a dropout of 0.3
        self.dropout = nn.Dropout(0.2)
        
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv_1(x)))
        x = self.pool(F.relu(self.conv_2(x)))
        x = self.pool(F.relu(self.conv_3(x)))
        x = self.pool(F.relu(self.conv_4(x)))
        x = self.pool(F.relu(self.conv_5(x)))

        
        # Flatten the input
        x = x.view(-1, 7*7*256)
        
        # Add the dropout
        x = self.dropout(x)
        
        # Add the linear layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

#-#-# Do NOT modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

model_scratch

Net(
  (conv_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=8, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [22]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            print(data)
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            optimizer.zero_grad()
            # Obtain the output from the model
            output = model(data)
            # Obtain loss
            loss = criterion(output, target)
            # Backward induction
            loss.backward()
            # Perform optimization step
            optimizer.step()            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            output = model(data)
            # Obtain the loss
            loss = criterion(output, target)
            # Add this loss to the list (same as before but instead of train we use valid)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))
            

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        if valid_loss < valid_loss_min:
            # Print an alert
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model..'.format(
                valid_loss_min,
                valid_loss))

            torch.save(model.state_dict(), save_path)
            
            # Update the new minimum
            valid_loss_min = valid_loss
        
    return model

In [32]:
model_scratch = train(
    20, 
    loaders_scratch, 
    model_scratch, 
    get_optimizer_scratch(model_scratch),
    criterion_scratch, 
    use_cuda, 
    'ignore.pt'
)

TypeError: Unexpected type <class 'numpy.ndarray'>